# Capstone - Where are the best neighborhoods to open up a restaurant in Calgary, Alberta, Canada?


In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

## 0) Webscrape Table from Wikipedia Page

In [2]:
#send get request
html_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_T')
soup = BeautifulSoup(html_data.content,'xml')

#identify html regarding the table, Inspect element of table on url to identify tags
table = soup.find('table',{'class':'wikitable sortable'})

In [3]:
#Find the Table Headers
table_headers = table.find_all('th')
list_headers = []
for header in table_headers:
    list_headers.append(header.text.strip()) #strip can remove the '\n'
print(list_headers)

#Grab the data for each row
table_rows = table.find_all('tr')
table_data = []
for row in table_rows:
    table_data.append([cell.text.strip() for cell in row.find_all('td')])

['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']


## 1) Pass Data into a Pandas DataFrame and begin Preprocessing

In [4]:
#Create the dataframe from the webscraped data (table_data and list_headers)
df = pd.DataFrame(table_data, columns=list_headers)
df.dropna(axis=0, inplace=True)
df 

,Postal Code,Borough,Neighborhood,Latitude,Longitude
1,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
2,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
3,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
4,T4A,Airdrie,East Airdrie,51.272450,-113.986980
5,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413
...,...,...,...,...,...
176,T5Z,Edmonton,West Lake District,53.5966,-113.4882
177,T6Z,Not assigned,Not assigned,Not assigned,Not assigned
178,T7Z,Stony Plain,Not assigned,53.5202,-114.0135
179,T8Z,Not assigned,Not assigned,Not assigned,Not assigned


### *1a) Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.*

In [5]:
#drop all rows where Borough is 'Not Assigned'
df_noNotAssigned = df[df['Borough'] != 'Not assigned']
print(df_noNotAssigned.shape)
df_noNotAssigned.head()

(144, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
1,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
2,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
3,T3A,Calgary,"Dalhousie, Edgemont, Hamptons, Hidden Valley",51.126060,-114.143158
4,T4A,Airdrie,East Airdrie,51.272450,-113.986980
5,T5A,Edmonton,"West Clareview, East Londonderry",53.5899,-113.4413


### *1b) More than one neighborhood can exist in one postal code area.*

The code below shows that this is no longer true for the dataset from the Wikipedia Page. 

Code is provided such that if the condition were true then the data would merge neighborhoods belonging to the same postal code and borough into a single row.

In [6]:
#Identify rows where Neighbourhood is 'Not Assigned'
df_Neighborhood_NotAssigned = df_noNotAssigned[df_noNotAssigned['Neighborhood']=='Not assigned']

print(df_Neighborhood_NotAssigned.shape)
#so there there are 45 rows where the Neighborhood is not assigned

df_Neighborhood_NotAssigned.head()

(45, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
7,T7A,Drayton Valley,Not assigned,53.2165,-114.9893
9,T9A,Wetaskiwin,Not assigned,52.9741,-113.3646
22,T4C,Cochrane,Not assigned,51.1896,-114.4774
27,T9C,Vegreville,Not assigned,53.4874,-112.0636
31,T4E,Red Deer County,Not assigned,52.2911,-113.7027


In [7]:
#if there were multiple neighborhoods belonging to the same postal code and borough then the following code would
#group the neighborhoods into the same row
df_UniquePostalCode = df_noNotAssigned.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
print(df_UniquePostalCode.shape)
df_UniquePostalCode.head()

(144, 3)


,Postal Code,Borough,Neighborhood
0,T1A,Medicine Hat,Central Medicine Hat
1,T1B,Medicine Hat,South Medicine Hat
2,T1C,Medicine Hat,North Medicine Hat
3,T1G,Taber,Not assigned
4,T1H,Lethbridge,North Lethbridge


In [8]:
len(df_noNotAssigned['Postal Code'].unique())
#this shows that after removing the rows where Borughs are 'Not Assigned', the number of unique postal codes is 144, 
#so the final dataframe with all the neighborhoods with the same postal code grouped together
#should have 144 rows.

144

### 1c) If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [9]:
#Identify rows that have boroughs but no assigned neighborhood
condition = df_UniquePostalCode['Neighborhood']=='Not assigned'
df_UniquePostalCode[condition].head()

,Postal Code,Borough,Neighborhood
3,T1G,Taber,Not assigned
7,T1L,Banff,Not assigned
8,T1M,Coaldale,Not assigned
9,T1P,Strathmore,Not assigned
10,T1R,Brooks,Not assigned


In [10]:
#For Neighborhoods that say 'Not Assigned', replace that with the Borough
df_UniquePostalCode.loc[condition, 'Neighborhood']=df_UniquePostalCode.loc[condition, 'Borough']

In [11]:
df_UniquePostalCode.head()

,Postal Code,Borough,Neighborhood
0,T1A,Medicine Hat,Central Medicine Hat
1,T1B,Medicine Hat,South Medicine Hat
2,T1C,Medicine Hat,North Medicine Hat
3,T1G,Taber,Taber
4,T1H,Lethbridge,North Lethbridge


In [12]:
df_UniquePostalCode.shape

(144, 3)

## 2) Adding in the Geospatial data (Latitude and Longitude)
Wikipedia already provides that info for each neighborhood so I will add that back in from the original dataframe

In [13]:
df_PCLatLong = df[['Postal Code', 'Latitude','Longitude']]
df_PCLatLong.head()

,Postal Code,Latitude,Longitude
1,T1A,50.036460,-110.679250
2,T2A,51.049680,-113.964320
3,T3A,51.126060,-114.143158
4,T4A,51.272450,-113.986980
5,T5A,53.5899,-113.4413


In [15]:
df_merged = df_UniquePostalCode.join(df_PCLatLong.set_index('Postal Code'),on='Postal Code')
df_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,T1A,Medicine Hat,Central Medicine Hat,50.036460,-110.679250
1,T1B,Medicine Hat,South Medicine Hat,50.0172,-110.651
2,T1C,Medicine Hat,North Medicine Hat,50.0556,-110.6822
3,T1G,Taber,Taber,49.7773,-112.158
4,T1H,Lethbridge,North Lethbridge,49.7118,-112.8196


In [16]:
#Reviewing rows of the dataset that might be missing information
df_merged[df_merged['Longitude'] == 'Not assigned']

,Postal Code,Borough,Neighborhood,Latitude,Longitude
16,T1Z,Rocky View,Rocky View,Not assigned,Not assigned
50,T3S,Calgary,Southeast Calgary,Not assigned,Not assigned
51,T3T,Tsuut'ina,Tsuut'ina,Not assigned,Not assigned
61,T4M,Blackfalds,Blackfalds,Not assigned,Not assigned
107,T6Y,Edmonton,South Industrial,Not assigned,Not assigned
127,T8T,Sturgeon County,Sturgeon County,Not assigned,Not assigned


In [17]:
df_merged[df_merged['Latitude'] == 'Not assigned']

,Postal Code,Borough,Neighborhood,Latitude,Longitude
16,T1Z,Rocky View,Rocky View,Not assigned,Not assigned
50,T3S,Calgary,Southeast Calgary,Not assigned,Not assigned
51,T3T,Tsuut'ina,Tsuut'ina,Not assigned,Not assigned
61,T4M,Blackfalds,Blackfalds,Not assigned,Not assigned
107,T6Y,Edmonton,South Industrial,Not assigned,Not assigned
127,T8T,Sturgeon County,Sturgeon County,Not assigned,Not assigned


In [18]:
MissingLatLong = list(df_merged[df_merged['Latitude'] == 'Not assigned'].Neighborhood)
print(MissingLatLong)

['Rocky View', 'Southeast Calgary', "Tsuut'ina", 'Blackfalds', 'South Industrial', 'Sturgeon County']


In [19]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#Get latitude and longitude for the neighborhood,Alberta
Province = 'AB'

for neighborhood in MissingLatLong:
    address = neighborhood+', '+Province

    geolocator = Nominatim(user_agent="Town_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(neighborhood, latitude, longitude))
    
    df_merged.loc[df_merged['Neighborhood'] == neighborhood,'Latitude'] = latitude
    df_merged.loc[df_merged['Neighborhood'] == neighborhood,'Longitude'] = longitude
    

The geograpical coordinate of Rocky View are 51.3699347, -114.014186.
The geograpical coordinate of Southeast Calgary are 50.851216050000005, -114.06823411395902.
The geograpical coordinate of Tsuut'ina are 50.96502825, -114.35042261318955.
The geograpical coordinate of Blackfalds are 52.3865007, -113.7831285922449.
The geograpical coordinate of South Industrial are 41.6249059, -85.90528913464351.
The geograpical coordinate of Sturgeon County are 53.8422302, -113.5406548.


In [20]:
#Review that all longitudes and latitudes have been assigned
MissingLatLong = list(df_merged[df_merged['Latitude'] == 'Not assigned'].Neighborhood)
print(MissingLatLong)

[]


# 3) Segmenting and Clustering Datapoints

In [21]:
#filter dataframe to only consider Boroughs which include the word 'Calgary'
df_CalgaryBors = df_merged[df_merged['Borough'].str.contains('Calgary')]
df_CalgaryBors.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
15,T1Y,Calgary,"Rundle, Whitehorn, Monterey Park",51.0759,-114.0015
17,T2A,Calgary,"Penbrooke Meadows, Marlborough",51.049680,-113.964320
18,T2B,Calgary,"Forest Lawn, Dover, Erin Woods",51.0318,-113.9786
19,T2C,Calgary,"Lynnwood Ridge, Ogden, Foothills Industrial, G...",50.9878,-114.0001
20,T2E,Calgary,"Bridgeland, Greenview, Zoo, YYC",51.0632,-114.0614
21,T2G,Calgary,"Inglewood, Burnsland, Chinatown, East Victoria...",51.0415,-114.0599
22,T2H,Calgary,"Highfield, Burns Industrial",50.9857,-114.0631
23,T2J,Calgary,"Queensland, Lake Bonavista, Willow Park, Acadia",50.9693,-114.0514
24,T2K,Calgary,"Thorncliffe, Tuxedo Park",51.0857,-114.0714
25,T2L,Calgary,"Brentwood, Collingwood, Nose Hill",51.0917,-114.1127


### 3a) Obtain venue information for the neighbourhoods

In [22]:
#access Foursquare API using my CLIENT_ID & CLIENT_SECRET stored as environmental variables 
#on my computer
import os

In [23]:
CLIENT_ID = os.environ.get('Foursquare_CLIENT_ID') # your Foursquare ID
CLIENT_SECRET = os.environ.get('Foursquare_CLIENT_SECRET') # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

In [24]:
#Function from IBM Data Science/Course 9/Week 3/ Lab 4a. Neighborhoods New York
#this function will take a series of names, latitudes, and longitudes (series of neighborhood geospatial data)
#and return a dataframe which includes venues (quantity depending on the limit variable) that are near the neighborhood lat & long

def getNearbyVenues(names, latitudes, longitudes, LIMIT=100, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
Calgary_venues = getNearbyVenues(names=df_CalgaryBors['Neighborhood'],
                                   latitudes=df_CalgaryBors['Latitude'],
                                   longitudes=df_CalgaryBors['Longitude']
                                  )

In [26]:
Calgary_venues.shape

(331, 7)

In [27]:
Calgary_venues.head(7)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rundle, Whitehorn, Monterey Park",51.0759,-114.0015,Shark Club Sports Bar & Grill,51.076126,-114.002560,Sports Bar
1,"Rundle, Whitehorn, Monterey Park",51.0759,-114.0015,Toad 'n' Turtle Pubhouse,51.076554,-114.002768,Gastropub
2,"Rundle, Whitehorn, Monterey Park",51.0759,-114.0015,Basha International Foods,51.072712,-114.000067,Grocery Store
3,"Rundle, Whitehorn, Monterey Park",51.0759,-114.0015,"Tony Roma's Ribs, Seafood, & Steaks",51.079903,-114.002528,BBQ Joint
4,"Rundle, Whitehorn, Monterey Park",51.0759,-114.0015,"Royal Hotel Calgary, Trademark Collection by W...",51.077599,-114.005072,Hotel
5,"Rundle, Whitehorn, Monterey Park",51.0759,-114.0015,Days Inn,51.072042,-114.000821,Hotel
6,"Rundle, Whitehorn, Monterey Park",51.0759,-114.0015,Tim Hortons,51.072578,-114.000743,Coffee Shop


In [28]:
Calgary_venues.groupby('Neighborhood').count().sort_values('Venue Category', ascending=False).head(15)

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Inglewood, Burnsland, Chinatown, East Victoria Park, Saddledome",56,56,56,56,56,56
"Connaught, West Victoria Park",41,41,41,41,41,41
"Oak Ridge, Haysboro, Kingsland, Kelvin Grove, Windsor Park",35,35,35,35,35,35
"City Centre, Calgary Tower",27,27,27,27,27,27
"Rosscarrock, Westgate, Wildwood, Shaganappi, Sunalta",21,21,21,21,21,21
"Bridgeland, Greenview, Zoo, YYC",20,20,20,20,20,20
"Millrise, Somerset, Bridlewood, Evergreen",16,16,16,16,16,16
"Mount Pleasant, Capitol Hill, Banff Trail",13,13,13,13,13,13
"Douglas Glen, McKenzie Lake, Copperfield, East Shepard",10,10,10,10,10,10


The table above provides an overview of the data available for each neighborhood(s). Recall that each row corresponds to an unique postal code for the city of Calgary, Alberta, Canada. These venue categories (as seen in the previous cell for 'Calgary_venues.head(7)') vary greatly and includes various businesses such as restaurants, service providers, and lodging. The Dataframe above sorts the neighborhood(s) such that those with more venues appear at the top.

In [31]:
# #CAN EXPORT TO EXCEL FOR FURTHER REVIEW
# Calgary_venues.to_excel('Calgary Venues -2020-08-02.xlsx')

## 3b) Analyze Each Neighborhood

In [32]:
#Check this city has no Venue Category as Neighborhood
Calgary_venues[Calgary_venues['Venue Category'].apply(lambda x:x.lower())=='neighborhood'] 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [33]:
#Read from the Venues.xlsx spreadsheet to obtain a dataframe of venue categories and their corresponding general category
df_GeneralCategory = pd.read_excel('Venues.xlsx',sheet_name='Sheet2',header=0,usecols="A:B")
df_GeneralCategory

,Venue Category,General Venue Category
0,American Restaurant,Food
1,Art Gallery,Entertainment
2,Arts & Crafts Store,Retail
3,Asian Restaurant,Food
4,Automotive Shop,Services
...,...,...
117,Vegetarian / Vegan Restaurant,Food
118,Video Store,Retail
119,Vietnamese Restaurant,Food
120,Wine Shop,Food


*Venue categories currently saved on the Venues.xlsx spreadsheet has been assigned a General Venue Category so that each venue can be compared to other venues in the same business sector.*

A list of the venues currently saved on the spreadsheet and the latest list of venue categories from Foursquare will be compared. Both lists will have all strings as lower case letters only so that no variations in upper/lower case letters identifies the same venue category as different venue categories.

Any new venues will be saved onto a third list so that it can be added to the spreadsheet and processed to identify the general category each new venue belongs to. 

In [34]:
#Obtain a list of venue categories with General categories already assigned from the Venues.xlsx spreadsheet
list_venueCategory_GENERAL = list(df_GeneralCategory['Venue Category'].apply(lambda x: x.lower()))

#Obtain a list of the venue categories returned by Foursquare for this City
list_CityVenueCategories = list(Calgary_venues['Venue Category'].apply(lambda x: x.lower())) 

#Create a list of new venue categories that are not currently on the Venues.xlsx spreadsheet
new_venueCategories =list(np.setdiff1d(list_CityVenueCategories,list_venueCategory_GENERAL))

#update the list again to match the title format of each venue category
new_venueCategories = [venue.title() for venue in new_venueCategories]
print(new_venueCategories)

['Carpet Store', 'Electronics Store', 'Forest', 'Insurance Office', 'Karaoke Bar', 'Lawyer', 'Skating Rink', 'Waterfront']


In [35]:
#Display a dataframe showing the new venue categories and their corresponding neighborhoods and venue names
New_Venues = Calgary_venues[Calgary_venues[['Venue Category']].isin(new_venueCategories).any(1)]
New_Venues = New_Venues[['Neighborhood', 'Venue', 'Venue Category']]
New_Venues

,Neighborhood,Venue,Venue Category
92,"Highfield, Burns Industrial",Stonegate Pub,Karaoke Bar
94,"Highfield, Burns Industrial",Fairview Arena,Skating Rink
98,"Queensland, Lake Bonavista, Willow Park, Acadia",Bow Valley Insurance,Insurance Office
104,"Thorncliffe, Tuxedo Park",Urban Piping Ltd,Waterfront
121,"Kensington, Westmont, Parkdale, University",Maguire Window Coverings,Carpet Store
264,"Douglas Glen, McKenzie Lake, Copperfield, East...",McKenna Park,Forest
297,"Rosscarrock, Westgate, Wildwood, Shaganappi, S...","Sandra G. Sebree, Lawyer",Lawyer
323,"Cranston, Auburn Bay, Mahogany",Telemaxx Communication Systems Ltd,Electronics Store


In [36]:
import openpyxl
import datetime

In [37]:
#if there are new venueCategories, these will be added to the bottom of the current Venues spreadsheet
if new_venueCategories: 
    wb = openpyxl.load_workbook('Venues.xlsx')
    ws = wb.worksheets[0]
    
    newRow = df_GeneralCategory.shape[0]+2 #this is the row on the excel where it will start appending the new venue categories
    
    #Add in something to show where the newest list begins
    timestamp = datetime.datetime.today().strftime('***Updated from here on %Y-%m-%d at %I-%M-%S %p')
    ws['C'+str(newRow)]=timestamp
    
    #Add in each of the new Venue Categories
    for venueCategory in new_venueCategories:
        A_cell = 'A'+str(newRow)
        ws[A_cell] = venueCategory
        newRow+=1
        
    wb.save('Venues.xlsx')

**User must open the spreadsheed 'Venues.xlsx'. New venue categories will be present at the bottom of the table. User must enter in the appropriate General Venue Category beside the new venue categories.**

General Venue Categories include:
- Bank
- Entertainment
- Food
- Health & Fitness
- Hospitality
- Retail
- Services

Foursquare has some venues that are not businesses. For example, there was a venue category titled intersection which can be set to '***To Delete' since that venue is not necessary for processing. 


The Excel sheet has been formatted as a table so it's recommended that the User clicks on the bottom right of blue table and expand the table boundaries to include the new data. Save the excel then return to the Jupyter Notebook.

In [38]:
# #Review the size of the original df_GeneralCategory
print('The original size was: ',df_GeneralCategory.shape)

#Update the df_GeneralCategory with the updated spreadsheet
df_GeneralCategory = pd.read_excel('Venues.xlsx',sheet_name='Sheet2',header=0,usecols="A:B")
print('The updated size is :',df_GeneralCategory.shape)

The original size was:  (122, 2)
The updated size is : (130, 2)


In [39]:
#Adding in the General Venue Category to the pandas dataframe
Calgary_venuesComb = Calgary_venues.join(df_GeneralCategory.set_index('Venue Category'), on='Venue Category')
Calgary_venuesComb.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,General Venue Category
0,"Rundle, Whitehorn, Monterey Park",51.0759,-114.0015,Shark Club Sports Bar & Grill,51.076126,-114.002560,Sports Bar,Food
1,"Rundle, Whitehorn, Monterey Park",51.0759,-114.0015,Toad 'n' Turtle Pubhouse,51.076554,-114.002768,Gastropub,Food
2,"Rundle, Whitehorn, Monterey Park",51.0759,-114.0015,Basha International Foods,51.072712,-114.000067,Grocery Store,Food
3,"Rundle, Whitehorn, Monterey Park",51.0759,-114.0015,"Tony Roma's Ribs, Seafood, & Steaks",51.079903,-114.002528,BBQ Joint,Food
4,"Rundle, Whitehorn, Monterey Park",51.0759,-114.0015,"Royal Hotel Calgary, Trademark Collection by W...",51.077599,-114.005072,Hotel,Hospitality


In [79]:
#Reduce dataframe such that there is one row for each General Venue Category for each Neighborhood(s)
#All the venues belong to each neighborhood(s) and each General Venue category is concantenated into one cell.
#This Dataframe is not used for the problem statement of this project but can be used for other purposes.
Calgary_AllVenues = Calgary_venuesComb.groupby(['Neighborhood','General Venue Category'])['Venue'].apply(', '.join).reset_index()
Calgary_AllVenues.head(15)

,Neighborhood,General Venue Category,Venue
0,"Braeside, Cedarbrae, Woodbine",Food,"Second Cup, Dairy Queen, Woody's Taphouse, Lit..."
1,"Braeside, Cedarbrae, Woodbine",Health & Fitness,"Fitness Plus, Ed Whalen Arena"
2,"Braeside, Cedarbrae, Woodbine",Retail,"Shoppers Drug Mart, 7-Eleven"
3,"Brentwood, Collingwood, Nose Hill",Health & Fitness,Brentwood Dog Park
4,"Bridgeland, Greenview, Zoo, YYC",Bank,"TD Canada Trust, ATB Financial"
5,"Bridgeland, Greenview, Zoo, YYC",Food,"Santorini Greek Taverna, Shikiji, Pho Kim Rest..."
6,"Bridgeland, Greenview, Zoo, YYC",Health & Fitness,"Calgary Climbing Centre Stronghold, McHugh Blu..."
7,"Bridgeland, Greenview, Zoo, YYC",Retail,"Shoppers Drug Mart, 7-Eleven, Don's Hobby Shop"
8,"City Centre, Calgary Tower",Entertainment,"Boudoir Rouge, Globe Cinema"
9,"City Centre, Calgary Tower",Food,"Gyu-Kaku Japanese BBQ, Holy Grill, Posto Pizze..."


In [40]:
###Project will be to review the best neighborhoods to start up a restaurant 
#so obtain dataframe with only the restaurants; General Venue Category = 'Food'
Filter = 'Food'

Calgary_Restaurants = Calgary_venuesComb[Calgary_venuesComb['General Venue Category'] == Filter][['Neighborhood','Venue','Venue Category']]
Calgary_Restaurants.head()

,Neighborhood,Venue,Venue Category
0,"Rundle, Whitehorn, Monterey Park",Shark Club Sports Bar & Grill,Sports Bar
1,"Rundle, Whitehorn, Monterey Park",Toad 'n' Turtle Pubhouse,Gastropub
2,"Rundle, Whitehorn, Monterey Park",Basha International Foods,Grocery Store
3,"Rundle, Whitehorn, Monterey Park","Tony Roma's Ribs, Seafood, & Steaks",BBQ Joint
6,"Rundle, Whitehorn, Monterey Park",Tim Hortons,Coffee Shop


In [41]:
#Refine Dataframe to count the number of restaurants within each neighborhood(s)
df_NumRestaurants = Calgary_Restaurants['Neighborhood'].value_counts().to_frame().reset_index()
df_NumRestaurants.columns = ['Neighborhood','# of Restaurants']
df_NumRestaurants.head()

,Neighborhood,# of Restaurants
0,"Inglewood, Burnsland, Chinatown, East Victoria...",38
1,"Connaught, West Victoria Park",30
2,"Oak Ridge, Haysboro, Kingsland, Kelvin Grove, ...",24
3,"City Centre, Calgary Tower",23
4,"Rosscarrock, Westgate, Wildwood, Shaganappi, S...",18


In [42]:
#add in the venue names for each of the restaurants within each neighborhood
df_right = Calgary_Restaurants.groupby('Neighborhood')['Venue'].apply(', '.join).reset_index()
df_NumRestaurants = df_NumRestaurants.join(df_right.set_index('Neighborhood'),on='Neighborhood')
df_NumRestaurants.columns = ['Neighborhood','# of Restaurants','Venues']
df_NumRestaurants.head()

,Neighborhood,# of Restaurants,Venues
0,"Inglewood, Burnsland, Chinatown, East Victoria...",38,"Grumans Catering and Delicatessen, 5 Vines Win..."
1,"Connaught, West Victoria Park",30,"Holy Grill, Posto Pizzeria and Bar, The Casbah..."
2,"Oak Ridge, Haysboro, Kingsland, Kelvin Grove, ...",24,"The Keg Steakhouse + Bar - Macleod Trail, Bole..."
3,"City Centre, Calgary Tower",23,"Gyu-Kaku Japanese BBQ, Holy Grill, Posto Pizze..."
4,"Rosscarrock, Westgate, Wildwood, Shaganappi, S...",18,"Moti Mahal Restaurant, Shawarma Knight, Cluck ..."


### 3c) Cluster Neighborhoods

In [43]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [44]:
# set number of clusters
kclusters = 5

CalgaryNumRestaurants_clustering = df_NumRestaurants.drop(['Neighborhood','Venues'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(CalgaryNumRestaurants_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 1, 1, 1, 3, 3, 3, 4, 4, 4])

In [45]:
# add clustering labels
df_NumRestaurants.insert(0, 'Cluster Labels', kmeans.labels_)
df_NumRestaurants.head()

,Cluster Labels,Neighborhood,# of Restaurants,Venues
0,2,"Inglewood, Burnsland, Chinatown, East Victoria...",38,"Grumans Catering and Delicatessen, 5 Vines Win..."
1,1,"Connaught, West Victoria Park",30,"Holy Grill, Posto Pizzeria and Bar, The Casbah..."
2,1,"Oak Ridge, Haysboro, Kingsland, Kelvin Grove, ...",24,"The Keg Steakhouse + Bar - Macleod Trail, Bole..."
3,1,"City Centre, Calgary Tower",23,"Gyu-Kaku Japanese BBQ, Holy Grill, Posto Pizze..."
4,3,"Rosscarrock, Westgate, Wildwood, Shaganappi, S...",18,"Moti Mahal Restaurant, Shawarma Knight, Cluck ..."


In [46]:
#Create a new Dataframe that includes cluster labels, Neighborhood, Neighborhood Latitude, Neighborhood Longitude,
#Number of Restaurants, and the Venue Names
Calgary_venuesCombCOPY = Calgary_venuesComb.copy()
Calgary_venuesCombCOPY = Calgary_venuesCombCOPY[Calgary_venuesCombCOPY['General Venue Category']=='Food']
Calgary_venuesCombCOPY = Calgary_venuesCombCOPY[['Neighborhood','Neighborhood Latitude','Neighborhood Longitude']]

#Check that the number of unique latitudes and longitudes match the number of neighborhood rows.
Neigh_lat = list(Calgary_venuesCombCOPY['Neighborhood Latitude'].unique())
Neigh_long = list(Calgary_venuesCombCOPY['Neighborhood Longitude'].unique())
Neighs = df_NumRestaurants.Neighborhood.unique().shape[0]
print('# of Neighborhood Latitudes:{}, Longitudes:{}, and the # of Neighborhoods:{}'.format(len(Neigh_lat),len(Neigh_long),Neighs))

# of Neighborhood Latitudes:27, Longitudes:27, and the # of Neighborhoods:27


In [47]:
df_NumRestaurants.insert(2, 'Neighborhood Longitude', Neigh_long)
df_NumRestaurants.insert(2, 'Neighborhood Latitude', Neigh_lat)
df_NumRestaurants.head()

,Cluster Labels,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,# of Restaurants,Venues
0,2,"Inglewood, Burnsland, Chinatown, East Victoria...",51.0759,-114.0015,38,"Grumans Catering and Delicatessen, 5 Vines Win..."
1,1,"Connaught, West Victoria Park",51.0318,-113.9786,30,"Holy Grill, Posto Pizzeria and Bar, The Casbah..."
2,1,"Oak Ridge, Haysboro, Kingsland, Kelvin Grove, ...",50.9878,-114.0001,24,"The Keg Steakhouse + Bar - Macleod Trail, Bole..."
3,1,"City Centre, Calgary Tower",51.0632,-114.0614,23,"Gyu-Kaku Japanese BBQ, Holy Grill, Posto Pizze..."
4,3,"Rosscarrock, Westgate, Wildwood, Shaganappi, S...",51.0415,-114.0599,18,"Moti Mahal Restaurant, Shawarma Knight, Cluck ..."


### *Visualize the Clusters*

In [48]:
#Get latitude and longitude for Calgary, AB
address = 'Calgary, AB'
geolocator = Nominatim(user_agent="Calgary_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Calgary, AB are 51.0534234, -114.0625892.


In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster,restNames,numRests in zip(df_NumRestaurants['Neighborhood Latitude'], 
                                                     df_NumRestaurants['Neighborhood Longitude'], 
                                                     df_NumRestaurants['Neighborhood'], 
                                                     df_NumRestaurants['Cluster Labels'],
                                                     df_NumRestaurants['Venues'],df_NumRestaurants['# of Restaurants']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster)+'\n Restaurants: '+str(restNames), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
#         radius=5,
        radius=int(numRests),
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3d) Examine Clusters

In [57]:
list_cols = ['Cluster Labels', 'Neighborhood', '# of Restaurants', 'Venues']

In [58]:
df_NumRestaurants.loc[df_NumRestaurants['Cluster Labels'] == 0, list_cols]

,Cluster Labels,Neighborhood,# of Restaurants,Venues
14,0,"Montgomery, Bowness, Silver Springs, Greenwood",3,"Boston Pizza, Starbucks, Market Mall Foodcourt"
15,0,"Douglas Glen, McKenzie Lake, Copperfield, East...",3,"Coasters, St. Vivians Irish Pub, Forever Good ..."
16,0,"Highfield, Burns Industrial",2,"Cravings Market Restaurant, Pita Queen"
17,0,"Dalhousie, Edgemont, Hamptons, Hidden Valley",2,"Edgemont City, Friends Cappuccino Bar & Bake Shop"
18,0,"Hawkwood, Arbour Lake, Citadel, Ranchlands, Ro...",2,"Al's Pizza, Stonewall Pub"
19,0,"Lynnwood Ridge, Ogden, Foothills Industrial, G...",2,"Ogden Pizza, Smokehouse Diner"
20,0,"Discovery Ridge, Signal Hill, West Springs, Ch...",2,"Schooner's Neighbourhood Pub, Treasures of Saigon"
21,0,"Queensland, Lake Bonavista, Willow Park, Acadia",2,"Golden Gate - Acadia, Gondola Pizza"
22,0,"Cranston, Auburn Bay, Mahogany",2,"Spirits of Cranston, Western Pizza"
23,0,"Lakeview, Glendale, Killarney, Glamorgan",2,"Tim Hortons / Esso, Highlander Wine & Spirits"


In [59]:
df_NumRestaurants.loc[df_NumRestaurants['Cluster Labels'] == 1, list_cols]

,Cluster Labels,Neighborhood,# of Restaurants,Venues
1,1,"Connaught, West Victoria Park",30,"Holy Grill, Posto Pizzeria and Bar, The Casbah..."
2,1,"Oak Ridge, Haysboro, Kingsland, Kelvin Grove, ...",24,"The Keg Steakhouse + Bar - Macleod Trail, Bole..."
3,1,"City Centre, Calgary Tower",23,"Gyu-Kaku Japanese BBQ, Holy Grill, Posto Pizze..."


In [60]:
df_NumRestaurants.loc[df_NumRestaurants['Cluster Labels'] == 2, list_cols]

,Cluster Labels,Neighborhood,# of Restaurants,Venues
0,2,"Inglewood, Burnsland, Chinatown, East Victoria...",38,"Grumans Catering and Delicatessen, 5 Vines Win..."


In [61]:
df_NumRestaurants.loc[df_NumRestaurants['Cluster Labels'] == 3, list_cols]

,Cluster Labels,Neighborhood,# of Restaurants,Venues
4,3,"Rosscarrock, Westgate, Wildwood, Shaganappi, S...",18,"Moti Mahal Restaurant, Shawarma Knight, Cluck ..."
5,3,"Bridgeland, Greenview, Zoo, YYC",14,"Santorini Greek Taverna, Shikiji, Pho Kim Rest..."
6,3,"Mount Pleasant, Capitol Hill, Banff Trail",11,"Carl's Jr., Jimmy's A&A Deli, Earl's Restauran..."


In [62]:
df_NumRestaurants.loc[df_NumRestaurants['Cluster Labels'] == 4, list_cols]

,Cluster Labels,Neighborhood,# of Restaurants,Venues
7,4,"Millrise, Somerset, Bridlewood, Evergreen",5,"Hudsons Shawnessy, Moxie's, Coach And Horses A..."
8,4,"Sandstone, MacEwan Glen, Beddington, Harvest H...",5,"Madison's Restaurant And Bar, Starbucks, Bosto..."
9,4,"Rundle, Whitehorn, Monterey Park",5,"Shark Club Sports Bar & Grill, Toad 'n' Turtle..."
10,4,"Tuscany, Scenic Acres",4,"Phoenix Kitchen + Lounge, Silver Springs Liquo..."
11,4,South Calgary (Altadore / Bankview / Richmond),4,"Famoso Neapolitan Pizzeria, Starbucks, M&M Foo..."
12,4,"Thorncliffe, Tuxedo Park",4,"McDonald's, Stavro’s Sports Bar, Pho Duy Thanh..."
13,4,"Braeside, Cedarbrae, Woodbine",4,"Second Cup, Dairy Queen, Woody's Taphouse, Lit..."
